# Creating a Custom Agent with MCP Tools in Azure AI

This notebook demonstrates the end-to-end process of creating a custom agent in Azure AI that utilizes a Microsoft Custom Copilot (MCP) tool. We will cover the following steps:
1.  **Setup & Configuration**: Authenticate and initialize the necessary clients.
2.  **Tool Definition**: Configure an MCP tool to connect to an external API.
3.  **Agent Creation**: Define and register a new agent with our custom tool.
4.  **Agent Interaction**: Engage in a conversation with the agent, handle the tool approval flow, and retrieve the final response.
5.  **Response Analysis**: Inspect the detailed output from the agent.


## 1. Prerequisites

Before you begin, ensure you have the following:
*   An Azure AI Project.
*   A deployed model within your project (e.g., a GPT-4 model).
*   The necessary permissions to create agents and interact with the project.
*   A `.env` file in the same directory as this notebook with the following variables:

```
PROJECT_ENDPOINT="<YOUR_AZURE_AI_PROJECT_ENDPOINT>"
MODEL_DEPLOYMENT_NAME="<YOUR_MODEL_DEPLOYMENT_NAME>"
```

In [1]:
import os
import time
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MCPTool, PromptAgentDefinition
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Setup and Configuration

First, we'll load the environment variables from our `.env` file. Then, we will create an `AIProjectClient` to manage resources within our Azure AI project and an `OpenAIClient` to interact with the conversational agent.


In [ ]:
# Load environment variables from .env file
load_dotenv()

# Retrieve configuration from environment variables
project_endpoint = os.environ.get('PROJECT_ENDPOINT')
model_deployment = os.environ.get('MODEL_DEPLOYMENT_NAME')

if not project_endpoint or not model_deployment:
    raise ValueError("Please set PROJECT_ENDPOINT and MODEL_DEPLOYMENT_NAME in your .env file.")

# Use DefaultAzureCredential for authentication
credential = DefaultAzureCredential()

# Create the main client to interact with the Azure AI Project
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

# Get the OpenAI-compatible client for conversational interactions
openai_client = project_client.get_openai_client()

print(f"Clients initialized for project endpoint: {project_endpoint}")

## 3. Define the MCP Tool

Here, we define the MCP tool configuration. The agent will use this tool to call an external API.

*   `server_label`: A unique identifier for the tool within the agent's configuration.
*   `server_url`: The URL pointing to the OpenAPI specification or the API endpoint for the MCP server.
*   `require_approval`: Determines if the agent must seek user approval before calling the tool. Options are `never`, `always`, or `auto`.


In [3]:
# Define the configuration for the MCP server tool
server_label = 'api-specs'
server_url = 'https://learn.microsoft.com/api/mcp' # This is a public Microsoft Learn API endpoint

mcp_tool = MCPTool(
    server_label=server_label,
    server_url=server_url,
    require_approval='always'  # 'never' allows the agent to call the tool without asking for permission
)

print(f"MCP Tool '{server_label}' configured for URL: {server_url}")


MCP Tool 'api-specs' configured for URL: https://learn.microsoft.com/api/mcp


## 4. Create the Agent

Now, we create the agent. We provide it with a name, a set of instructions defining its persona and purpose, the model it should use, and the list of tools it has access to.


In [4]:
# Define the agent's properties
agent_name = "learn-doc-agent"
agent_instructions = """
You are a helpful assistant designed to find relevant documentation on Microsoft Learn.
Use the provided MCP tools to answer user questions about Microsoft products and services.
Prefer calling the MCP tools when they can improve the accuracy of your answer.
"""

# Create a new version of the agent
my_agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment,
        instructions=agent_instructions,
        tools=[mcp_tool]
    ),
)

print(f"Agent created successfully!")
print(f"  ID: {my_agent.id}")
print(f"  Name: {my_agent.name}")
print(f"  Version: {my_agent.version}")


Agent created successfully!
  ID: learn-doc-agent:1
  Name: learn-doc-agent
  Version: 1


## 5. Interact with the Agent

With our agent created, we can now start a conversation.

### 5.1. Create a Conversation Thread
First, we create a new conversation thread, which will maintain the context of our interaction.


In [9]:
# Create a new conversation thread for the agent
conversation = openai_client.conversations.create()
print(f"Created a new conversation thread with ID: {conversation.id}")


Created a new conversation thread with ID: conv_3efd6544ad8a3f3300uWuoy7IzMuMkch5WymGRR0V945irbwoi


### 5.2. Send the Initial Request
We send our first message. The agent will analyze this request and determine that it needs to use the configured MCP tool to answer it. Because we set `require_approval='never'`, it will proceed directly. If it were set to `always` or `auto`, the flow would pause here for an approval step.


In [10]:
# Send the initial user request to the agent
user_prompt = "Find one relevant Microsoft Learn page about Machine Learning Model creation"

response = openai_client.responses.create(
    conversation=conversation.id,
    input=user_prompt,
    extra_body={"agent": {"name": my_agent.name, "type": "agent_reference"}},
)

print("Initial request sent to the agent.")
print(f"Agent response: {response.output_text}")


Initial request sent to the agent.
Agent response: 


## 6. Analyze the Response

Let's create a helper function to print the components of the agent's response object. This helps us understand the steps the agent took, including any tool calls it made.


In [11]:
def pretty_print_response(resp):
    """A helper function to print the agent's response object in a readable format."""
    print("=" * 20)
    print("AGENT RESPONSE DETAILS")
    print("=" * 20)
    for i, item in enumerate(resp.output):
        print(f"\n--- Item {i+1}: type = {item.type} ---")
        if getattr(item, "text", None):
            # Limit text output to avoid flooding the screen
            print(f"  Text: {item.text[:500]}...")
        if getattr(item, "role", None):
            print(f"  Role: {item.role}")
        if getattr(item, "server_label", None):
            print(f"  Server Label: {item.server_label}")
        if item.type == "tool_response":
            print(f"  Tool Name: {item.name}")
            print(f"  Arguments: {item.arguments}")
            print(f"  Output: {item.output[:500]}...")

# Print the detailed response from the last interaction
pretty_print_response(response)


AGENT RESPONSE DETAILS

--- Item 1: type = mcp_list_tools ---
  Server Label: api-specs

--- Item 2: type = mcp_approval_request ---
  Server Label: api-specs
